<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing I</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=522">MICRO-511</a>) taught by Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2020.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:pol.delaguilapla@epfl.ch">Pol del Aguila Pla</a>, 
    <a href="mailto:kay.lachler@epfl.ch">Kay Lächler</a>,
    <a href="mailto:alejandro.nogueronaramburu@epfl.ch">Alejandro Noguerón Arámburu</a>, and
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
</p>
<hr style="clear:both">
<h1>Lab 1.2: The Fourier transform</h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: Thursday October 7, 2021</p>
    <p style="margin:4px;"><b>Submission</b>: <span style="color:red">Friday October 15, 2021</span> (before 11:59PM) on <a href="https://moodle.epfl.ch/course/view.php?id=522">Moodle</a></p>
    <p style="margin:4px;"><b>Grade weigth</b>: Lab 1 (16 points), 9% of the overall grade</p>
    <p style="margin:4px;"><b>Remote help</b>: Monday October 11, on Zoom (see Moodle for link and time)</p>    
    <p style="margin:4px;"><b>Related lectures</b>: Chapters 1 and 2</p>
</div>

### Student Name: Guanqun Liu
### SCIPER: 334988

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [1]:
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

SCIPER: 334988


### Imports
In the next cell we import the python libraries that we will use throughout the lab, as well as the `ImageViewer` class (Python package developed specifically for these laboratories, see documentation [here](https://github.com/Biomedical-Imaging-Group/interactive-kit/wiki/Image-Viewer), or run the python command `help(viewer)` after loading the class):
* [`matplotlib.pyplot`](https://matplotlib.org), to display images,
* [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/), to make the image display interactive, and
* [`numpy`](https://numpy.org/doc/stable/reference), for mathematical operations on arrays.

Finally, we load the images used in this lab. Note that we will use the [OpenCV](https://opencv.org/) library for loading the images, and then we will make sure that it is of type `float64`, for higher accuracy during operations. **Note that, while in the pixelwise operations lab we didn't need high accuracy, for transforms like the Fourier transform, as well as other transforms you will see in [IP2](https://moodle.epfl.ch/course/view.php?id=463), it is essential to have the highest accuracy available.**

Run the next cell to get your notebook ready.
<div class=" alert alert-danger">
    
<i>Note:</i> Always run the two import cells below before starting to work on the notebook.    
</div>

In [2]:
# Configure plotting as dynamic
%matplotlib widget

# Import required packages for this lab
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import cv2 as cv
from interactive_kit import imviewer as viewer

# Loading images
joux = cv.imread("images/joux.tif", cv.IMREAD_UNCHANGED).astype('float64')
car = cv.imread("images/car_pad.tif", cv.IMREAD_UNCHANGED).astype('float64')
mandrill = cv.imread("images/mandrill.tif", cv.IMREAD_UNCHANGED).astype('float64')
impulse = np.zeros((65,65)); impulse[32,32] = 1;
pens = cv.imread("images/pens.tif", cv.IMREAD_UNCHANGED).astype('float64')
zebra = cv.imread("images/zebra.tif", cv.IMREAD_UNCHANGED).astype('float64')

# The Fourier transform (7 points)

In this second part we will look at the 2D discrete Fourier transform (abbreviated as *DFT*), what it represents, and how an image can be reconstructed from its 2D discrete Fourier transform by using the inverse discrete Fourier transform (*iDFT*). First, you will:
 * understand the effects of the elements in an image on its Fourier transform (*FT*), and then
 * understand how an image is reconstructed from its FT using the inverse Fourier transform (*iFT*). 

In this part of the lab we will only use Python, since implementing a *FT* in a low level language is beyond the scope of this course. To compute the FT in Python, we will use the [`fft` module](https://numpy.org/doc/stable/reference/routines.fft.html) in NumPy, which implements the *FT* using a [fast Fourier transform (FFT)](https://en.wikipedia.org/wiki/Fast_Fourier_transform) algorithm.

## <a id="ToC_2_Fourier_transform"></a>Table of contents

1. [Understanding the Fourier transform and its inverse](#1.-Understanding-the-Fourier-transform-and-its-inverse-(4-points)) **(4 points)**
2. [Reconstruction of an image](#2.-Reconstruction-of-an-image-(3-points)) **(3 points)**
    1. [Reconstruction error](#2.A.-Reconstruction-error-(1-point))
    2. [Fourier components](#2.B.-Fourier-components-(1-point))

Take some time to explore the images you will be using. Run the next cell and look at the histograms, the range of values, etc.
<div class="alert alert-info">
    
<b>Hint:</b> Use the buttons <code>Prev</code>/<code>Next</code> to cycle through the images.
</div>

In [3]:
# Define the list of images
image_list = [joux, car, mandrill, impulse, pens, zebra]
# Display all images used in this lab
initial_viewer = viewer(image_list, hist = True)

# 1. Understanding the Fourier transform and its inverse (4 points)
[Back to table of contents](#ToC_2_Fourier_transform)

First, we will provide the functions `fourier(img)` and `inverse_fourier(ft)`, which calculate the FT and the IFT respectively.

<div class="alert alert-info">
<b>Note:</b> We will make use of the following <i>NumPy</i> functions:
<ul>
<li><a href='https://numpy.org/doc/stable/reference/generated/numpy.fft.fft2.html'><code>np.fft.fft2(img)</code></a> calculates the two-dimensional FT of <code>img</code>,</li>
<li><a href='https://numpy.org/doc/stable/reference/generated/numpy.fft.fftshift.html#numpy.fft.fftshift'><code>np.fft.fftshift(ft)</code></a> shifts the frequency range of the FT <code>ft</code> from $[0, \pi]$ to $[-\frac{\pi}{2}, \frac{\pi}{2}]$,</li>
<li><a href='https://numpy.org/doc/stable/reference/generated/numpy.fft.ifft2.html#numpy.fft.ifft2'><code>np.fft.ifft2(ft)</code></a> calculates the inverse FT of the two-dimensional FT <code>ft</code>.</li>
</ul>in the wrappers <code>fourier</code> and <code>inverse_fourier</code>. Make sure to understand how we use the <code>numpy.fft</code> module.
</div>

In [4]:
# Function that returns the FT
def fourier(img):
    # Generate the FT
    ft = np.fft.fft2(img)
    # Shift the frequency range to [-pi/2, pi/2] 
    shift_ft = np.fft.fftshift(ft)
    return shift_ft

# Function that return the inverse FT
def inverse_fourier(ft):
    # Shift the FT back to [0, pi]
    ft = np.fft.ifftshift(ft)
    # Get the inverse FT
    ift = np.fft.ifft2(ft)
    # Clip the imaginary part of the reconstruction
    # (should be approximately zero anyway)
    ift = np.real(ift)
    return ift

As you know, calculating the FT of an image generates a two-dimensional array (image) of complex values, which makes it challenging to find a good visualization. Therefore, we usually extract the **magnitude** and **phase** of the complex numbers, which are much easier to deal with and present useful information. Remember that the magnitude of a complex number $z\in\mathbb{C}$ is given by
$$|z| = \sqrt{\operatorname{Re}(z)^2+\operatorname{Im}(z)^2}.$$

Furthermore, we usually visualize this magnitude in dB, i.e., 

$$|z|~[\mathrm{dB}] = 10\log_{10}\left(|z|^2\right).$$

One of the reasons for this is that the variations in the magnitude of the Fourier transform generally span very different ranges, from the very small to the very large, and the $\log(\cdot)$ transformation allows us to visualize both in the same image.

To do this, you will first define the function `magnitude(ft)`, which should return **the magnitude in decibels (`dB`)** of a FT given as an input parameter. For **1 point**, complete the function `magnitude(ft)` in the cell below according to the equation given above.

<div class="alert alert-info">

<b>Hints:</b>

<li> With NumPy, you can extract the <b>real</b> part of a complex variable <code>z</code> using <a href="https://numpy.org/doc/stable/reference/generated/numpy.real.html" ><code>np.real(z)</code></a> and the <b>imaginary</b> part using <a href="https://numpy.org/doc/stable/reference/generated/numpy.imag.html" ><code>np.imag(z)</code></a>,</li>
<li> if you need it, use <a href="https://numpy.org/doc/stable/reference/generated/numpy.sqrt.html" ><code>np.sqrt(x)</code></a> to get the square root of <code>x</code>,</li>
<li> use <a href="https://numpy.org/doc/stable/reference/generated/numpy.log10.html" ><code>np.log10(x)</code></a> to get the base-10 logarithm of <code>x</code>.</li>
</div>

<div class="alert alert-danger">
<b>Warning:</b> Using <code>np.absolute</code> in this exercise will give you <b>0 points</b>! We want you to implement the function yourself.
</div>

In [9]:
# Function that returns the magnitude of the FT in dB
def magnitude(ft):
    # Initialize the output to 0
    output = None
    
    # Apply the magnitude function of ft, and transform the result into dB
    output = 10 * np.log10(np.power(np.real(ft), 2) + np.power(np.imag(ft), 2))
    
    # Return the output
    return output

In [10]:
# Let's do a sanity check
# The complex number used for the test which has a magnitude of ~3 dB
z = 1 + 1j
# Check that the magnitude function is correct
if np.round(magnitude(z), decimals=1) == 3.0:
    print("Nice, your magnitude function passed the basic sanity check!")
else:
    print("Something isn't quite right yet.")

Nice, your magnitude function passed the basic sanity check!


Now, we will define a function to calculate the phase of the *FT*. For this we define the function `phase(ft)`, which takes as argument an *FT* and returns its phase.

Remember that the phase of a complex number $z$ is given by
$$\angle(z)=\arctan\left(\frac{\operatorname{Im}(z)}{\operatorname{Re}(z)}\right)\,.$$

For **0.5 points**, complete the function `phase(ft)` in the cell below according to the equation given above.

<div class="alert alert-info">
<b>Hint:</b> Use <a href="https://numpy.org/doc/stable/reference/generated/numpy.arctan2.html" ><code>np.arctan2(x1, x2)</code></a> to calculate the $\arctan\left(\frac{x_1}{x_2}\right)$. This function even chooses the corresponding quadrant for you, so you do not need to check for negative angles.
</div>
<div class="alert alert-danger">
<b>Warning:</b> Using <code>np.angle</code> in this exercise will give you <b>0 points</b>! Implement the function yourself.
</div>

In [11]:
# Function that calculates the phase of complex numbers
def phase(ft):
    # Initialize output variable
    output = None
    
    # YOUR CODE HERE
    output = np.arctan2(np.imag(ft), np.real(ft))
    
    return output

In [12]:
# Let's do a sanity check
# The complex number used for the test which has a phase of pi/4
z = 1 + 1j
# Check that the magnitude function is correct
if phase(z) == np.pi/4:
    print("Nice, your phase function passed the sanity check!")
else:
    print("Something isn't quite right yet.")

Nice, your phase function passed the sanity check!


Now, let's look at the resuts of the functions you just coded. For this we will apply the *FT* to the image `car`, calculate its magnitude and phase, and visualize the results as images. 

Run the next cell to see the magnitude and phase of the *FT* of `car`.

</div>
<div class="alert alert-success">
<b>Hint:</b> If you don't see the phase of the image, use the sliding bar to slide to the right. You can also use <code>Ctrl + b</code> to hide the left sidebar of JupyterLab. 

If everything went well you should see:
<ul><li>For the magnitude: a diagonal cross in the center with stars spread over the image, and</li>
<li>For the phase: random noise, cut by a near vertical line and a few other straight lines.</li>
</ul>
</div>

In [13]:
# Generate the FT of car
car_ft = fourier(car)
# Calculate the magnitude
car_ft_mag = magnitude(car_ft)
# Calculate the phase
car_ft_ph = phase(car_ft)
# Visualize the two together with the original image
plt.close('all')
ft_vis = viewer([car, car_ft_mag, car_ft_ph], title=['Car', 'Car FT magnitude', 'Car FT phase'], subplots=(1,3))

Button(description='Show Widgets', style=ButtonStyle())

### Multiple Choice Questions 
The following questions will test your understanding of the relationship of an image with its FT **magnitude**. The first two will ask about the image `car`, and the second two will ask about the image `pens`. Each MC question is worth **0.5 points**. As usual, we will include a cell for you to change your answer (in the variable `answer`) and a cell to check that you chose one of the possible answers. 

Run the next cell to visualize the images `pens` and `car` together with their FT magnitudes and answer the upcoming questions.

In [14]:
 # Get the FT magnitudes of the images using the fourier and magnitude functions
car_ft  = magnitude(fourier(car))
pens_ft = magnitude(fourier(pens))

# Define the lits of images and names
image_list = [car, car_ft, pens, pens_ft]
title_list = ['Car', 'FT of Car', 'Pens', 'FT of Pens']

# Display results
plt.close('all')
ft_viewer = viewer(image_list, title=title_list, subplots = [2, 2], colorbar = True)

Button(description='Show Widgets', style=ButtonStyle())

* Q1: Where do the little stars at different distances from the center in the FT of `car` come from?
    1. From the contour of the car.
    2. From the driver.
    3. From the carpet under the car.
    4. From the details of the car (JAGUAR text, doors, steering wheel, etc).
    5. From the size of the image.

In the next cell, modify the variable `answer` to reflect your choice.

In [23]:
# Assign your answer to this variable
answer = 3
# YOUR CODE HERE

In [24]:
assert answer in [1, 2, 3, 4, 5], 'Possible answers are 1, 2, 3, 4 and 5'

* Q2: Where do the two big intersecting lines in `car` come from? 
    1. From the contour of the car.
    2. From the driver.
    3. From the carpet under the car.
    4. From the details of the car (JAGUAR text, doors, steering wheel, etc).
    5. From the size of the image.

In the next cell, modify the variable `answer` to reflect your choice.

In [25]:
# Assign your answer to this variable
answer = 4
# YOUR CODE HERE

In [26]:
assert answer in [1, 2, 3, 4, 5], 'Valid answers are 1, 2, 3, 4, and 5'

* Q3: Why is there only one main line in the FT of `pens`, if there are two pens?
    1. Because the background is constant.
    2. Because they are ballpoint pens and not fountain pens.
    3. Because the two pens are aligned.
    4. Because the two pens are close to each other.

In the next cell, modify the variable `answer` to reflect your choice.

In [27]:
# Assign your answer to this variable
answer = 3
# YOUR CODE HERE

In [28]:
assert answer in [1, 2, 3, 4], 'Valid answers are 1, 2, 3 or 4'

* Q4: Why is the main line in the *FT* not aligned with the pens? 

    1. The frequency of a contour is perpendicular to the contour. 
    2. The main periodicity in the image is *parallel* to the pens because they have rough surfaces.
    3. The `viewer` rotates the FT.
    4. Numpy rotates the FT.

In the next cell, modify the variable `answer` to reflect your choice.

In [29]:
# Assign your answer to this variable
answer = 1
# YOUR CODE HERE

In [30]:
assert answer in [1, 2, 3, 4, 5], 'Valid answers are 1, 2, 3, 4 and 5'

# 2. Reconstruction of an image (3 points)
[Back to table of contents](#ToC_2_Fourier_transform)

As you learned in the course, it is possible to reconstruct an image from its Fourier transform by performing the inverse Fourier transform. In the next exercise we will investigate the role that the magnitude and phase of the *FT* has on the reconstruction of an image. For this we first need to create a function that reconstructs an image from its *FT* magnitude (in $\mathrm{dB}$) and phase. Run the next cell to define the function `reconstruct` and make sure that you understand every line of the code.

In [31]:
# Function that reconstructs an image from its FT magnitude (in dB) and phase
def reconstruct(mag, ph):
    # Since the magnitude is in dB we first need to convert it back
    mag = 10 ** (mag / 20)
    # Now we can restore the complex FT from the magnitude and phase using the polar representation
    ft = mag * np.exp(1j * ph)
    # Having the complex FT we can simply use the inverse_fourier function that we defined above to reconstruct the image
    return inverse_fourier(ft)

Let's see if the function works. Run the cell below to reconstruct the car image from its magnitude and phase, and visualize the result. 

<div class = 'alert alert-warning'>
<b>Warning:</b> If the reconstruction is not near perfect, check again your functions <code>magnitude</code> and <code>phase</code>. You will need both functions to answer the next questions. 
</div>

In [32]:
# Reconstruct the car image
car_reconstructed = reconstruct(car_ft_mag, car_ft_ph)
# Display the result
plt.close('all')
ft_rec_vis = viewer([car, car_reconstructed], title=['Original car', 'Reconstructed car'], subplots=(1,2))
np.testing.assert_array_almost_equal(car, car_reconstructed, err_msg='Check again your magnitude and phase functions!')

Button(description='Show Widgets', style=ButtonStyle())

Since we didn't make any changes to the *FT* before the reconstruction, the reconstructed image should be (almost) identical to the original image (if it's not you should have seen an error message). 

For the next excercise we will use the `mandrill` image in addition to the car image. Run the next cell to visualize it again. Moreover, we will plot its *FT*'s magnitude and phase (browse through the images with the buttons `Next` & `Prev`). This will help you to answer the upcoming questions.

In [33]:
# Generate FT of the mandrill image
mandrill_ft = fourier(mandrill)
# Extract the magnitude and phase
mandrill_ft_mag = magnitude(mandrill_ft)
mandrill_ft_ph = phase(mandrill_ft)

# Visualize
plt.close('all')
mandrill_vis = viewer([mandrill, mandrill_ft_mag, mandrill_ft_ph], widgets=True)

Now lets see what happens if we use the magnitude of one image and the phase of another image to do the reconstruction. What do you think will happen? Run the cell below and observe the results. Try to make a conclusion on what type of information is stored in the phase of the *FT*.

In [34]:
# Reconstruct an image with the magnitude of car and phase of mandrill
car_mandrill = reconstruct(car_ft_mag, mandrill_ft_ph)
# Reconstruct an image with the magnitude of mandrill and phase of car
mandrill_car = reconstruct(mandrill_ft_mag, car_ft_ph)
# Visualize the results
plt.close('all')
rec_comp_vis = viewer([car_mandrill, mandrill_car], title=['Magn. = car, Phase = mandrill', 'Magn. = mandrill, Phase = car'], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

### Multiple choice question

What type of information of the image is stored in the **phase** of the FT that is **not stored** in the magnitude? (**0.5 points**)

1. The spacial frequencies contained in the image.
2. The light intensity of each pixel.
3. The location and shape of objects in the image.

Modify the variable `answer`in the next cell to reflect your choice. The second cell is for you to check that you have entered a valid answer.

In [35]:
# Assign your answer to this variable
answer = 3
# YOUR CODE HERE

In [36]:
# Sanity check
assert answer in [1, 2, 3], 'Valid answers are 1, 2 or 3'

## 2.A. Reconstruction error (1 point)
[Back to table of contents](#ToC_2_Fourier_transform)

How many Fourier coefficients do we really need to keep to still have the basic information present in an image? Do all coefficients contribute the same? In order to answer this type of questions, it is good to define an objective metric of _how good_ a certain reconstruction is. Here, we will discuss the normalized mean square error (NMSE) to reconstruct an image $f$. This metric assesses how different an image $f$ is from its reconstruction $g$, but normalizes it by the total power of $f$. In particular, we have that if $K$ and $L$ are the number of columns and rows, respectively,

$$\operatorname{NMSE}_f(g) =  \frac{\sum_{k=1}^{K} \sum_{l=1}^L (g[k,l] - f[k,l])^2}{\sum_{m=1}^{K} \sum_{n=1}^L f[m,n]^2}\mbox{, and } \operatorname{NMSE}_f(g)~[\mathrm{dB}] = 10 \log_{10}\left(\operatorname{NMSE}_f(g)\right).$$

As we can see, the NMSE is often expressed in dB. This makes it easier for one to observe, for example, when the error has doubled ($+3~\mathrm{dB}$) or halved ($-3~\mathrm{dB}$) in plots.

For **1 point**, complete the function `nmse(f, g)` in the cell below according to the equation given above, where `f` and `g` are two NumPy arrays of the same shape.
<div class="alert alert-info">
<b>Hints:</b> Use <a href="https://numpy.org/doc/stable/reference/generated/numpy.mean.html" ><code>np.mean(x)</code></a> to calculate the mean value of <code>x</code>, or 
    <a href="https://numpy.org/doc/stable/reference/generated/numpy.sum.html" ><code>np.sum(x)</code></a> to calculate its sum through all axis.
</div>
<div class="alert alert-danger">
<b>Warning</b>: Remember that your function should <b>not use <code>for</code> loops</b> to iterate through images (this will give you $0$ points), and should work for NumPy arrays of any shape.
</div>

In [41]:
# check = np.ones((3,3))
# np.sum(check)

In [42]:
# Function that calculates the Normalized Mean Square Error in dB
def nmse(f, g):
    # Declare the output variable
    output = None
    
    # YOUR CODE HERE
    nmse = np.sum(np.power((g - f), 2)) / np.sum(np.power(f, 2))
    output = 10 * np.log10(nmse)
    
    # Return MSE
    return output

In [43]:
# Sanity check (do not worry about the divide by zero note)
if  nmse(impulse, impulse) != -np.infty: 
    print('The error between two equal images should be zero. In dB -infinity.')
# Check your function on the hrct image
elif nmse(impulse, 0) != 0:
    print('The error of any image and a zero-image should be 1. In dB, 0.')
else:
    print('Nice, your function seems to work! Do not worry about the divide by zero warning!')

Nice, your function seems to work! Do not worry about the divide by zero warning!


/tmp/ipykernel_40/2835904627.py:8: RuntimeWarning: divide by zero encountered in log10
  output = 10 * np.log10(nmse)


## 2.B. Fourier components (1 point)
[Back to table of contents](#ToC_2_Fourier_transform)

In this section we look into the reconstruction process of an image from part of its Fourier components. This touches on a topic that will continue to appear in IP1 and IP2: how much does a given transform compress an image? 

For a first practical approximation to the topic, we define the method `clip_fourier(img, percent)`. This method reconstructs an image for only `percent`$\%$ of its Fourier coefficients. If `largest=True`, only the largest are kept, while 
if `largest=False`, only all the rest are kept. This will illustrate the uneven distribution of information contained in the Fourier components.

Run the next cell to define the function `clip_fourier`.

In [44]:
 def clip_fourier(img, percent, largest=True, perc = True):
    # Get number of coefficients to keep
    if perc:
        # Extract from percentage
        n = np.round(np.prod(img.shape) * percent / 100 ).astype(int)
    else: 
        # Pass directly
        n = percent
    # Get ft of img
    img_ft = np.fft.fft2(img)
    # Get the threshold value. To do this, we order the Fourier coefficients 
    # from low to high and select the n-to-last ([-n]) coefficient
    threshold = np.sort(np.abs(img_ft.flatten()))[-n]
    if largest == True:
        # Get the inverse Fourier transform of the thresholded Fourier transform
        clipped_ift = np.real(np.fft.ifft2((np.abs(img_ft) >= threshold) * img_ft))
    else:
        # Get the inverse Fourier transform of the thresholded Fourier transform
        clipped_ift = np.real(np.fft.ifft2((np.abs(img_ft) < threshold) * img_ft))
    return clipped_ift

Let's use the error metric `nmse` defined before to illustrate the difference in information contained in the few largest Fourier components compared to the information contained in the rest. In the cell below we will reconstruct the image `zebra` using: 
 * only the `percent` largest Fourier components, and 
 * using the `100-percent` smallest components. 

Then we will compare the reconstruction error by applying the `nmse` function defined above with both reconstructions. Run the cell below to see the different reconstruction errors. Play with the variable `percent` and see what happens.

In [45]:
percent = 20
# First, reconstruct zebra using the largest components
zebra_largest = clip_fourier(zebra, percent)
# Reconstruct zebra using the smallest components
zebra_smallest = clip_fourier(zebra, percent, largest=False)
# Calculate the errors
error_l = nmse(zebra,zebra_largest )
error_s = nmse(zebra,zebra_smallest)
# Compare the error
print(f'The reconstruction error when using the {percent}% largest  components: NMSE = {error_l:.4f}')
print(f"The reconstruction error when using the {100 - percent}% smallest components: NMSE = {error_s:6.4f}")
# Visualize images
view = viewer([zebra, zebra_largest, zebra_smallest], 
              titles = ['Original', f'{percent}% Largest Components', f'{100-percent}% Smallest Components'], 
              widgets=True)

The reconstruction error when using the 20% largest  components: NMSE = -23.7228
The reconstruction error when using the 80% smallest components: NMSE = -0.0185


In the long run, this type of characteristics of transforms are explored using graphs like the one below, where the NMSE can be seen as a function of the percentage of the largest coefficients kept.

In [46]:
plt.close("all")
# Create figure
fig = plt.figure(num=f"SCIPER: {uid}",figsize = (8, 5));
# Plot the NMSE vs kept coefficients curve
plt.plot( np.arange(.5,100,5), [nmse(zebra, clip_fourier(zebra, perc)) for perc in np.arange(.5,100,5)], 'bo-' );
# Labels and titles for clear plotting
plt.xlabel("Percentage of coefficients kept"); plt.ylabel("NMSE [dB]"); 
plt.xticks([0,20,40,60,80,100], [f"{perc}%" for perc in [0,20,40,60,80,100]]);
plt.title("Reconstructing zebra with the largest Fourier coefs.")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…

Now we are going to create a widget to apply the function to an image and dinamically visualize its effect. 

We will define a slider to choose an integer `n` such that **the number of largest coefficients kept is $2^n$**. 
<div class = "alert alert-info">
<b>Note</b>: This is because the visual difference between the reconstructions is only apparent for percentages between $0\%$ and $2\%$, and very small steps would be needed. Keep in mind that you are not working with percentages anymore.
    
</div>

We will also provide a checkbox to switch between the two modes of operation (keeping the largest, or keeping all the rest). Click the button `Apply` to apply `clip_fourier()` with the chosen parameter on the image. Run the next cell and click on `Extra Widgets` to use the widget. Explore the results carefuly.

In [47]:
# Declare slider and checkbox
n_slider = widgets.IntSlider( value = 16, min = 0, max = 16, step = 1, description = 'n' )
checkbox = widgets.Checkbox(value=True, description='Use largest components')

# Declare btutton
button = widgets.Button(description = 'Apply')

# declare the button callback
def button_callback(image):
    n      = n_slider.value
    check  = checkbox.value
    output = clip_fourier(image, 2**n, largest = check, perc = False)
    return output

# visualize
plt.close('all')
cfourier_viewer = viewer(zebra, title = "Clipping the FT", new_widgets = [n_slider, checkbox, button], callbacks = [button_callback], widgets=True, normalize=True)

### Multiple Choice Question
Congratulations! You made it to the end of the notebook. Now you just need to answer these last two MCQ questions (**0.5 points** each).

* Q1: How many largest Fourier coefficients are required to start noting a zebra shape in the image?
    1. 1
    2. 8
    3. $2^8$
    4. $8^2$

Modify the variable `answer` to reflect your choice. The second cell will raise an error if you have not answered.

In [48]:
# assign your answer to this variable
answer = 2
# YOUR CODE HERE

In [49]:
assert answer in [1, 2, 3, 4], 'Possible answers are 1, 2, 3 and 4'

* Q2: How is it possible to reconstruct the zebra from only periodic components if the zebra is not periodic (there is only one zebra in the image)?
    1. The black and white stripes in the zebra make it possible.
    2. The FT assumes that the image is periodic in space.
    3. The biggest components of the FT are non-periodic, to account for such features in an image.

Modify the variable `answer` to reflect your choice. The second cell will raise an error if you have not answered.

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
assert answer in [1, 2, 3], 'Possible answers are 1, 2, and 3'

<div class="alert alert-success">
<p><b>Congratulations on finishing the second part of the Pixel-Fourier lab!</b></p>
<p>
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to <a href="https://moodle.epfl.ch/mod/assign/view.php?id=1157357">Moodle</a>, in a zip file with the other notebook of this lab.
</p>
</div>

* Keep the name of the notebook as: *2_Fourier_transform.ipynb*,
* Name the zip file: *Pixel_Fourier_Lab.zip*.